In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import config
import numpy as np
from train import FaceGen


if __name__ == "__main__":
    #misc.init_output_logging()
    np.random.seed(config.common.random_seed)

   
    print('Running FaceGen()...')
    facegen = FaceGen()
    facegen.train()
    
    print('Exiting...')
    

Running FaceGen()...
********** New Layer [4 x 4] : batch_size 64 **********
from_it 1, total_it 390


TypeError: expected np.ndarray (got str)

In [15]:
from util.scheduler import LevelScheduler
import numpy as np


training_size = 400
transition_size = 800

for i in range(0, 2000, 100) :
    sched = LevelScheduler(i, training_size, transition_size)
    print("cur image : %d, stats %s, level %.3f" % (i, sched.state, sched.level))

cur image : 0, stats train, level 1.000
cur image : 100, stats train, level 1.000
cur image : 200, stats train, level 1.000
cur image : 300, stats train, level 1.000
cur image : 400, stats transition, level 1.000
cur image : 500, stats transition, level 1.125
cur image : 600, stats transition, level 1.250
cur image : 700, stats transition, level 1.375
cur image : 800, stats transition, level 1.500
cur image : 900, stats transition, level 1.625
cur image : 1000, stats transition, level 1.750
cur image : 1100, stats transition, level 1.875
cur image : 1200, stats train, level 2.000
cur image : 1300, stats train, level 2.000
cur image : 1400, stats train, level 2.000
cur image : 1500, stats train, level 2.000
cur image : 1600, stats transition, level 2.000
cur image : 1700, stats transition, level 2.125
cur image : 1800, stats transition, level 2.250
cur image : 1900, stats transition, level 2.375


In [3]:
"""
Noticable things.
1. png files could look bluish. no problem when using cv2 imread/imwrite.
-> if bothers u, use cv2.cvtColor/cv2.COLOR_RGB2BGR
2. file name could be attr combination.
"""

import os
import argparse

import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


def parse_tfrecord_np(tfr_file):
    ex = tf.train.Example()
    ex.ParseFromString(tfr_file)
    shape = ex.features.feature['shape'].int64_list.value
    data = ex.features.feature['data'].bytes_list.value[0]
    attr = ex.features.feature['attr'].bytes_list.value[0]
    return np.fromstring(data, np.uint8).reshape(shape), np.fromstring(attr, np.uint8)


def extract_tfrecord_np(tfr_files, save_dir):
    for tfr_file in tfr_files:
        tfr_opt = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.NONE)
        i = 0
        for record in tf.python_io.tf_record_iterator(tfr_file, tfr_opt):
            img, attr = parse_tfrecord_np(record)
            img = img.transpose(1, 2, 0)
            res = str(img.shape[1])
            img_file_name = ''.join([str(i) for i in attr]) + f'_{i}.png'
            img_file_path = os.path.join(save_dir, res, img_file_name)
            plt.imsave(img_file_path, img)
            i += 1


def make_sub_dirs(save_dir):
    res = [2**i for i in range(2, 11)]
    for i in res:
        dir_path = os.path.join(save_dir, str(i))
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)


def load_tfrecord_from_dir(tfr_dir):
    return sorted(glob.glob(tfr_dir + '/tfrecord*'))


print("Start")

tfr_dir='E:/sjyoon/homework/face-completion-with-controllable-attributes-via-pggan/test_data/celebA/tfrecord/'
make_sub_dirs(tfr_dir)
tfr_files = load_tfrecord_from_dir(tfr_dir)
extract_tfrecord_np(tfr_files, tfr_dir)
print("End")


Start
Make Dirs


C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


End


In [1]:
import torch
import numpy as np

attr_real = torch.rand(1, 4)

attr_real[attr_real < 0.5] = 0
attr_real[attr_real >= 0.5] = 1

print(attr_real)
        
N, attr_size = attr_real.shape
attr_obs = attr_real.clone()

batch_index = np.arange(N)
batch_index = batch_index[np.random.rand(N) > 0.5]

attr_index = np.random.randint(attr_size, size=len(batch_index))

print(batch_index)
print(attr_index)


attr_obs[batch_index, attr_index ] = 1 - attr_real[batch_index, attr_index]
print(attr_obs)

tensor([[ 0.,  0.,  0.,  1.]])
[0]
[0]
tensor([[ 1.,  0.,  0.,  1.]])
